In [5]:
!nvidia-smi

Wed Jun 23 14:48:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 98% 1.02G/1.04G [00:04<00:00, 233MB/s]
100% 1.04G/1.04G [00:04<00:00, 233MB/s]


In [9]:
!rm -r Image-Captioning/

In [10]:
# # get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 506, done.
remote: Counting objects: 100% (506/506), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 506 (delta 312), reused 475 (delta 281), pack-reused 0
Receiving objects: 100% (506/506), 36.49 MiB | 20.79 MiB/s, done.
Resolving deltas: 100% (312/312), done.


In [11]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [12]:
# !pip3 install nltk==3.6.2
# import nltk
# nltk.download("wordnet")

In [13]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from torch.utils.tensorboard import SummaryWriter
from os import path as osp

In [14]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 20  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
pretrained_embeddings = False
fine_tune_embeddings = False
checkpoint = None  # path to checkpoint, None if none

In [15]:
DATA_NAME = 'flickr8k_tb'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/data.json'
IMGS_PATH = 'flickr8k/images/'

In [16]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)
vocab_len = len(vocab)
vocab_len

100%|██████████| 40000/40000 [00:00<00:00, 301842.41it/s]


5089

In [17]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

logger_dic = {
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
    'max_seq_length': 100,
    'vocab_size': vocab_len,
    'enocder': 'resnet101',
    'dropout': dropout,
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim 
    
}

In [18]:
t_params

{'batch_size': 256,
 'data_name': 'flickr8k_tb',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 20,
 'fine_tune_encoder': False,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': False,
 'vocab': <dataset.Vocabulary at 0x7f3b3bba8490>,
 'workers': 2}

In [19]:
# experiment name
name = DATA_NAME + "lr_scheduler"
# path
log_dir = 'experiments'

logger = SummaryWriter(log_dir=osp.join(log_dir, name))

In [20]:
# with scheduler
fit(t_params=t_params, m_params=m_params, logger=logger)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth



Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/118]	Batch Time 9.028 (9.028)	Data Load Time 3.641 (3.641)	Loss 9.4521 (9.4521)	Top-5 Accuracy 0.063 (0.063)
Epoch: [0][100/118]	Batch Time 2.481 (2.466)	Data Load Time 0.001 (0.041)	Loss 4.9276 (5.5162)	Top-5 Accuracy 53.816 (44.561)
Epoch train time 290.046 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 5.779 (5.779)	Loss 5.4145 (5.4145)	Top-5 Accuracy 49.337 (49.337)	
----- Bleu-n Scores -----
1: 58.0121116989451
2: 32.0200032972207
3: 14.738920682083856
4: 6.612428994577319
-------------------------

 * LOSS - 5.379, TOP-5 ACCURACY - 50.760, BLEU-4 - 6.612428994577319

Epoch validation time 53.770 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [1][0/118]	Batch Time 5.989 (5.989)	Data Load Time 3.340 (3.340)	Loss 4.8646 (4.8646)	Top-5 Accuracy 52.797 (52.797)
Epoch: [1][100/118]	Batch Time 2.455 (2.520)	Data Load Time 0.001 (0.038)	

In [21]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_tb.pth.tar'
epochs = 30

t_params['batch_size'] = batch_size
t_params['data_name'] = t_params['data_name'] + "_finetune" 
t_params['fine_tune_encoder'] = True
t_params['epochs'] = epochs
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_tb_finetune',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': False,
 'vocab': <dataset.Vocabulary at 0x7f3b3bba8490>,
 'workers': 2}

In [ ]:
fit(t_params, checkpoint=checkpoint, m_params=m_params, logger=logger)

Loaded Checkpoint!!
Starting Epoch: 10
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [10][0/469]	Batch Time 4.587 (4.587)	Data Load Time 0.882 (0.882)	Loss 3.7892 (3.7892)	Top-5 Accuracy 68.785 (68.785)
Epoch: [10][100/469]	Batch Time 1.476 (1.386)	Data Load Time 0.002 (0.010)	Loss 4.0912 (4.0430)	Top-5 Accuracy 63.774 (64.056)
Epoch: [10][200/469]	Batch Time 1.338 (1.370)	Data Load Time 0.001 (0.006)	Loss 3.7205 (3.9978)	Top-5 Accuracy 70.263 (64.732)
Epoch: [10][300/469]	Batch Time 1.336 (1.366)	Data Load Time 0.000 (0.004)	Loss 3.9416 (3.9762)	Top-5 Accuracy 67.085 (65.154)
Epoch: [10][400/469]	Batch Time 1.295 (1.363)	Data Load Time 0.001 (0.003)	Loss 3.8530 (3.9615)	Top-5 Accuracy 68.275 (65.37

In [ ]:
!cp /content/BEST_checkpoint_flickr8k_tb_finetune.pth.tar /content/drive/MyDrive/ImageCaptioning/flickr8
!cp /content/experiments /content/drive/MyDrive/ImageCaptioning/flickr8